In [ ]:
import json
import os
from glob import glob

data_root = "c:/Users/Noh/github/Accident_Prediction_Prevent/Models/Object_Detection/sample_data/" # 상위 디렉토리

json_dir = data_root + "test_data/라벨링데이터_231108_add/차대보행자_이미지_육교및지하도부근/bb_1_161018_pedestrian_112_331/" # 모든 JSON 파일이 저장된 폴더
source_dir = data_root + "test_data/원천데이터_231108_add/차대보행자_이미지_육교및지하도부근/bb_1_161018_pedestrian_112_331/" # 원본 이미지 파일이 저장된 폴더

output_json_file = data_root + "coco_annotations.json"  # JSON 파일을 병합하여 저장할 파일명

In [2]:
# COCO 데이터셋 초기화
coco_format = {
    "images": [],
    "annotations": [],
    "categories": []
}

# 카테고리 매핑 (고유한 ID를 유지하기 위해 사용)
category_mapping = {"two-wheeled-vehicle" : 0, 
                    "traffic-light-red" : 1, 
                    "pedestrian" : 2, 
                    "crosswalk" : 3, 
                    "bike" : 4,
                    "traffic-light-green" : 5,
                    "vehicle" : 6,
                    "traffic-light-etc" : 7,
                    "traffic-sign" : 8}

annotation_id = 1

# 초기화된 category 정보를 COCO 형식에 추가
for category_name, category_id in category_mapping.items():
    coco_format["categories"].append({
        "id": category_id,
        "name": category_name
    })

# JSON 파일 목록 가져오기
json_files = glob(os.path.join(json_dir, "*.json"))  # 모든 JSON 파일 검색
print(f"총 {len(json_files)}개의 JSON 파일을 처리합니다.")

# 모든 JSON 파일 읽고 COCO 형식으로 변환
for json_file in json_files:
    with open(json_file, "r") as f:
        data = json.load(f)

    file_name = data["file_name"]
    width, height = data["width"], data["height"]
    image_id = data["sequence_frame_number"]

    # 이미지 정보 추가
    coco_format["images"].append({
        "file_name": source_dir + file_name,
        "width": width,
        "height": height,
        "id": image_id
    })

    # 객체 정보 추가
    for obj in data["objects"]:
        category_name = obj["category"]
        bbox = obj["bbox"]  # [x, y, width, height]

        # 새로운 카테고리 발견 시 추가
        if category_name not in category_mapping:
            category_mapping[category_name] = category_id
            coco_format["categories"].append({
                "id": category_id,
                "name": category_name
            })
            category_id += 1

        # 객체 주석 추가
        coco_format["annotations"].append({
            "bbox": bbox,
            "area": bbox[2] * bbox[3],  # width * height
            "iscrowd": 0,
            "image_id": image_id,
            "category_id": category_mapping[category_name],
            "id": annotation_id,
        })

        annotation_id += 1

    image_id += 1  # 이미지 ID 증가

# COCO JSON 파일 저장
with open(output_json_file, "w") as f:
    json.dump(coco_format, f, indent=4)

print(f"COCO JSON 변환 완료! 저장된 파일: {output_json_file}")


총 118개의 JSON 파일을 처리합니다.
COCO JSON 변환 완료! 저장된 파일: c:/Users/Noh/github/Accident_Prediction_Prevent/Models/Object_Detection/sample_data/coco_annotations (2).json


In [3]:
json_files

['c:/Users/Noh/github/Accident_Prediction_Prevent/Models/Object_Detection/sample_data/test_data/라벨링데이터_231108_add/차대보행자_이미지_육교및지하도부근/bb_1_161018_pedestrian_112_331\\331_112_001.json',
 'c:/Users/Noh/github/Accident_Prediction_Prevent/Models/Object_Detection/sample_data/test_data/라벨링데이터_231108_add/차대보행자_이미지_육교및지하도부근/bb_1_161018_pedestrian_112_331\\331_112_002.json',
 'c:/Users/Noh/github/Accident_Prediction_Prevent/Models/Object_Detection/sample_data/test_data/라벨링데이터_231108_add/차대보행자_이미지_육교및지하도부근/bb_1_161018_pedestrian_112_331\\331_112_004.json',
 'c:/Users/Noh/github/Accident_Prediction_Prevent/Models/Object_Detection/sample_data/test_data/라벨링데이터_231108_add/차대보행자_이미지_육교및지하도부근/bb_1_161018_pedestrian_112_331\\331_112_005.json',
 'c:/Users/Noh/github/Accident_Prediction_Prevent/Models/Object_Detection/sample_data/test_data/라벨링데이터_231108_add/차대보행자_이미지_육교및지하도부근/bb_1_161018_pedestrian_112_331\\331_112_006.json',
 'c:/Users/Noh/github/Accident_Prediction_Prevent/Models/Object_Detection/sample

In [24]:
from pycocotools.coco import COCO
import json

# COCO 데이터셋 불러오기
coco_gt = COCO(output_json_file)

# COCO 데이터셋의 image_id 목록
coco_image_ids = set(coco_gt.getImgIds()) # 1 ~ 118까지 저장

# 예측 JSON 파일 로드
json_files = glob(os.path.join(json_dir, "*.json"))  # 모든 JSON 파일 검색
for json_file in json_files:
    with open(json_file, "r") as f:
        coco_pred = json.load(f)

    # 예측 JSON의 image_id 목록
    pred_image_ids = set([coco_pred["sequence_frame_number"]])

    coco_image_ids -= pred_image_ids # 코코에서 현재 이미지 제거

print(f"현재 존재하지 않는 이미지 아이디 : {coco_image_ids}")

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
현재 존재하지 않는 이미지 아이디 : set()
